# Importing your own neural net

Verifying the example neural net was all well and good, but you probably want to verify your own neural net now. In this tutorial, we show you how to import the parameters for the feed-forward net in the introduction.

In [ ]:
using MIPVerify
using Gurobi
using MAT

We'll download a `.mat` file containing the parameters of the sample neural net, containing two layers (exported from `tensorflow`). 

In [ ]:
param_dict = Base.download("https://github.com/vtjeng/MIPVerify_data/raw/master/weights/mnist/n1.mat") |> matread

## Layer 1

Let's begin by importing the parameters for the first fully connected layer, which has 784 inputs (corresponding to a flattened 28x28 image) and 24 outputs.

### Basic Approach

We begin with a basic approach where we extract the weights and the biases of the fully connected layer seperately.

In [ ]:
fc1_weight = param_dict["fc1/weight"]

In [ ]:
fc1_bias = param_dict["fc1/bias"]

We group the weights and biases in a `Linear`.

_(NB: We have to flatten the bias layer using `dropdims` since `Linear` expects a 1-D array for the bias.)_

In [ ]:
fc1_manual = Linear(fc1_weight, dropdims(fc1_bias, dims=1))

That was a lot to remember. Wouldn't it be nice if there was a helper function to take care of all that?

### With Helper Functions

In [ ]:
fc1 = get_matrix_params(param_dict, "fc1", (784, 40))

`get_matrix_params` requires that 1) you specify the expected size of the layer, and 2) your weight and bias arrays following the naming convention outlined in the [documentation](https://vtjeng.github.io/MIPVerify.jl/stable/utils/import_weights.html#MIPVerify.get_matrix_params-Tuple{Dict{String,V} where V,String,Tuple{Int64,Int64}}).

As a sanity check, you can verify that the parameters we get from both methods are equal.

In [ ]:
fc1.matrix == fc1_manual.matrix

In [ ]:
fc1.bias == fc1_manual.bias

## Importing the rest of the layers

Since we followed the naming convention required by `get_matrix_params` when exporting our neural net parameters as a `.mat` file, importing the rest of the neural net is relatively straightforward.

In [ ]:
fc2 = get_matrix_params(param_dict, "fc2", (40, 20))

In [ ]:
logits = get_matrix_params(param_dict, "logits", (20, 10))

## Composing the network

We now put the entire network together. We need to flatten the input since the input images are provided as a 4-dimensional tensor. (Note that there is no `ReLU` after the final softmax layers).

In [ ]:
n1 = Sequential([
        Flatten(4),
        fc1,
        # you can always use interval arithmetic for the first layer
        ReLU(interval_arithmetic),
        fc2,
        ReLU(),
        logits
    ], "MNIST.n1")

There we go! Now it's your turn to try to verify your own neural network.